### BootCamp Santader - Explorando IA Generativa em um Pipeline de ETL com Python

In [71]:
url_api = 'https://sdw-2023-prd.up.railway.app'

#### Extract
###### Extração dos Dados 
###### Utilizando pandas e a biblioteca requests iremos extrair dados da API.

In [72]:
import pandas as pd

df = pd.read_csv("Users.csv")
user_ids = df["id"].to_list()
user_ids

[1, 2, 4, 5, 6]

##### Fazendo uma requisição a API

In [73]:
import requests
import json

def get_user(id):
    response = requests.get(f"{url_api}/users/{id}")
    return response.json() if response.status_code == 200 else None 


users = [user for id in user_ids if(user:= get_user(id)) is not None]
print(json.dumps(users, indent=2))


[
  {
    "id": 1,
    "name": "Devweekerson",
    "account": {
      "id": 1,
      "number": "01.097954-4",
      "agency": "2030",
      "balance": 624.12,
      "limit": 1000.0
    },
    "card": {
      "id": 1,
      "number": "xxxx xxxx xxxx 1111",
      "limit": 2000.0
    },
    "features": [
      {
        "id": 2,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
        "description": "Pagar"
      },
      {
        "id": 3,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
        "description": "Transferir"
      },
      {
        "id": 4,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
        "description": "Conta Corrente"
      },
      {
        "id": 5,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/cards.svg",
        "description": "Cart\u00f5es"
      },
     

### Transform
#### Transformação e Melhoramento a Partir da API do Chat GPT

Aqui eu criei uma mensagem de um gerente para o cliente.

In [76]:
import openai

openai.api_key = "SUA API OPENAI"

def gerar_msg_gerente(user):
    completion = openai.ChatCompletion.create(
  model = 'gpt-3.5-turbo',
  messages = [
    {'role': 'system', 
     'content': 'Você é o gerente de um grande banco brasileiro! Seu nome é Riquinho das finanças e trabalha no Santander'},
    {
     'role':'user',
     'content':f'Crie apenas oferta direcionada para cada cliente: {user["name"]}, separe cada oferta e deixe uma saudação para cada cliente.'
    }
  ]
)
    return completion['choices'][0]['message']['content']


for user in users:
    msg = gerar_msg_gerente(user)
    user['features'].append(
        {
          "icon": "Riquinho das finanças: GitHub: hvenanc",
          "description": msg
        }
    )

users[1]


{'id': 5,
 'name': 'Pip',
 'account': {'id': 8,
  'number': '00002-2',
  'agency': '0001',
  'balance': 0.0,
  'limit': 500.0},
 'card': {'id': 5, 'number': '**** **** **** 2222', 'limit': 1000.0},
 'features': [{'icon': 'Riquinho das finanças: GitHub: hvenanc',
   'description': 'Oferta para o Pip:\n\nOlá Pip, é um prazer estar em contato com você e poder oferecer nossos serviços bancários exclusivos. Como cliente Santander, você já desfruta de benefícios únicos, mas gostaríamos de te apresentar uma oferta especial: um pacote de investimentos diversificados, especialmente criado para maximizar seus ganhos. Com essa oferta, você terá a oportunidade de ter uma carteira de investimentos personalizada, com opções que atendam ao seu perfil e objetivos financeiros. Aproveite essa oportunidade para fazer seu dinheiro trabalhar para você.\n\nSaudações,\nRiquinho das Finanças\n\n_________________________________________________\n\nOferta para o Separador de ofertas:\n\nOlá Separador de ofertas

### Load

#### Carregando as novas informações para a API

In [75]:
def load_data(user):
    response = requests.put(f"{url_api}/users/{user['id']}", json=user)
    return response.status_code

for user in users:
    sucesso = load_data(user)
    print(sucesso)


422
500
500
